In [3]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

from tensorflow.keras.models import Model



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import pandas as pd

In [9]:
# load data 

BASE_DIR ='D:/agriculture_computer_vision/Agriculture-Vision/train/images/'

rgb= os.path.join(BASE_DIR, 'rgb/')

cloud_shadow = os.path.join(BASE_DIR, 'cloud_shadow')

standing_water = os.path.join(BASE_DIR, 'standing_water')

double_plant = os.path.join(BASE_DIR,'double_plant')

planter_skip = os.path.join(BASE_DIR + 'planter_skip')

water_way = os.path.join(BASE_DIR,'waterway')

weed_cluster = os.path.join(BASE_DIR, 'weed_cluster')





#test dir
TEST_DIR = 'D:/agriculture_computer_vision/Agriculture-Vision/test/images/'
test_set = os.path.join(TEST_DIR, 'smaller_test_set')

VAL_DIR = 'D:/agriculture_computer_vision/Agriculture-Vision/val/images/rgb/'

In [ ]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 20
data_augmentation = True
num_classes = 6


input_shape = (256, 256, 3)

In [ ]:
# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

In [ ]:
n = 3

In [ ]:
# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

In [ ]:
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version) 

In [ ]:
train_df = pd.read_csv(r'D:\agriculture_computer_vision\final_encoded.csv')
train_df.drop(columns='Unnamed: 0', inplace=True)
#strip train index
train_df['index'] = train_df['index'].str.rstrip('.png')
train_df['index'] = train_df['index'] + '.jpg'

columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

In [ ]:
train_df

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.10) # set validation split

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df, 
    directory = 'D:/agriculture_computer_vision/Agriculture-Vision/train/images/rgb/',
    x_col = "index",
    y_col = columns,
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(256, 256),
    subset='training'
) # set as training data



validation_generator = train_datagen.flow_from_dataframe(
    dataframe= train_df,
    directory = 'D:/agriculture_computer_vision/Agriculture-Vision/train/images/rgb/',
    x_col = "index",
    y_col = columns,
    batch_size=32, 
    seed=42,
    shuffle = True,
    class_mode="raw",
    target_size = (256, 256),
    subset = 'validation')

In [ ]:
from keras import regularizers, optimizers

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(256,256,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))
model.compile(optimizers.Adam(lr=0.0001, decay=1e-6),loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# checkpoint
filepath="50_epochs_best_adam.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
#### STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
VAL_STEP_SIZE = validation_generator.n//validation_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = validation_generator,
                    epochs=50, callbacks=callbacks_list
                   )

In [ ]:
model.save("adam_2.h5")

In [1]:
from keras.models import load_model

model = load_model(r'C:\Users\User\Desktop\agri_vision\50_epochs_best_adam.h5')

Using TensorFlow backend.
C:\Users\User\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorboard\compat\t

In [12]:
val_df = pd.read_csv(r'C:\Users\User\Desktop\agri_vision\val_encoded.csv')

val_df.drop(columns='Unnamed: 0', inplace=True)

#strip train index
val_df['index'] = val_df['index'].str.rstrip('.png')

val_df['index'] = val_df['index'] + '.jpg'

columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

In [13]:
val_df['index']

0       18X67NLZ2_1219-1091-1731-1603.jpg
1       18X67NLZ2_1219-1603-1731-2115.jpg
2        18X67NLZ2_1219-579-1731-1091.jpg
3        18X67NLZ2_707-1603-1219-2115.jpg
4       1J7JL2PDK_1130-1579-1642-2091.jpg
                      ...                
4426     ZKV2W31CE_739-4212-1251-4724.jpg
4427     ZKV2W31CE_777-3820-1289-4332.jpg
4428     ZKV2W31CE_887-3539-1399-4051.jpg
4429     ZKV2W31CE_951-2419-1463-2931.jpg
4430     ZKV2W31CE_957-2884-1469-3396.jpg
Name: index, Length: 4431, dtype: object

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255) # set validation split

val_generator = train_datagen.flow_from_dataframe(
    dataframe = val_df, 
    directory = 'D:/agriculture_computer_vision/Agriculture-Vision/val/images/rgb/',
    x_col = "index",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(256, 256),
)

Found 4431 validated image filenames.


In [15]:
preds = model.predict(val_generator)

In [16]:
preds

array([[3.8921672e-01, 4.6506554e-02, 3.0054529e-03, 1.5997516e-02,
        1.4912057e-02, 1.2352863e-01],
       [8.2327408e-01, 5.3218052e-02, 1.1437878e-02, 4.0780008e-02,
        6.2501766e-03, 1.3880223e-01],
       [3.3109450e-01, 6.7535758e-02, 8.0447393e-03, 5.2881297e-03,
        2.9980965e-02, 2.1595469e-01],
       ...,
       [1.1022639e-05, 4.9200449e-02, 2.9016814e-05, 9.1662374e-04,
        4.8873339e-02, 9.9507207e-01],
       [1.5889005e-03, 2.5290055e-02, 1.3964329e-03, 2.3270727e-03,
        3.8091545e-03, 9.6007818e-01],
       [1.0912654e-04, 1.6684210e-02, 2.9025602e-04, 2.3996332e-03,
        1.0010702e-02, 9.8759127e-01]], dtype=float32)

In [17]:
y_val_true = val_df.drop(columns='index').values.argmax(axis=1)
y_val_pred = preds.argmax(axis=1)

In [18]:
# Importing the metrics package from sklearn library
from sklearn import metrics
# Creating the confusion matrix
cm = metrics.confusion_matrix(y_val_true, y_val_pred)

In [19]:
# Assigning columns names
cm_df = pd.DataFrame(cm, 
            columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster'],
            index = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster'])
# Showing the confusion matrix
cm_df

,cloud_shadow,double_planter,planter_skip,standing_water,waterway,weed_cluster
cloud_shadow,113,8,0,13,2,73
double_planter,0,172,0,3,1,264
planter_skip,1,6,0,2,0,14
standing_water,10,3,0,99,1,176
waterway,1,4,0,3,30,473
weed_cluster,35,23,0,39,20,2842


In [20]:
metrics.accuracy_score(y_val_true, y_val_pred)

0.734822839088242

In [21]:
target_names =  ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

from sklearn.metrics import classification_report
print(classification_report(y_val_true, y_val_pred, target_names=target_names))

                precision    recall  f1-score   support

  cloud_shadow       0.71      0.54      0.61       209
double_planter       0.80      0.39      0.52       440
  planter_skip       0.00      0.00      0.00        23
standing_water       0.62      0.34      0.44       289
      waterway       0.56      0.06      0.11       511
  weed_cluster       0.74      0.96      0.84      2959

      accuracy                           0.73      4431
     macro avg       0.57      0.38      0.42      4431
  weighted avg       0.71      0.73      0.68      4431



C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
